In [26]:
#Boston House pricing verisetinde cross validation yaparaak en iyi LinearRegrression parametrelerini bulacağız.
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder
spark = SparkSession.Builder().appName('Spark DataFrame Introduction').getOrCreate()
houseDF = spark.read.option('inferSchema','true')\
                        .option('header','true')\
                        .option('delimiter',',').csv('datasets/boston-house-prices.txt')
houseDF = houseDF.withColumnRenamed('medv','label')
houseDF.show()
#houseDF.describe().show()
vec = VectorAssembler(inputCols=houseDF.columns[0:-1],outputCol='features')
houseDF=vec.transform(houseDF)
houseDF = houseDF.select('features','label')
#houseDF.show(truncate=False)
lr = LinearRegression()
trainDF, testDF = houseDF.randomSplit([0.8,0.2])
model = lr.fit(trainDF)
resultDF = model.transform(testDF)
eva = RegressionEvaluator(metricName='rmse')
res = eva.evaluate(resultDF)
print('RMSE : ',res)

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+-----+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio|     b|lstat|label|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+-----+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98| 24.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14| 21.6|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03| 34.7|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94| 33.4|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33| 36.2|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21| 28.7|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5605|  5|311|   15.2| 395.6|12.43| 22.9|
|0.14455|12.5| 7.87|   0|0.524|6.172| 96.1|5.9505|  5|311|   15.2| 396.9|19.15| 27.1|
|0.21124|12.5| 7.87|   0|0.524|5.631|100.0|6.0821|  5|

In [25]:
#Boston House pricing verisetinde cross validation yaparaak en iyi LinearRegrression parametrelerini bulacağız.
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder
spark = SparkSession.Builder().appName('Spark DataFrame Introduction').getOrCreate()
houseDF = spark.read.option('inferSchema','true')\
                        .option('header','true')\
                        .option('delimiter',',').csv('datasets/boston-house-prices.txt')
houseDF = houseDF.withColumnRenamed('medv','label')
#houseDF.describe().show()
vec = VectorAssembler(inputCols=houseDF.columns[0:-1],outputCol='features')
houseDF=vec.transform(houseDF)
houseDF = houseDF.select('features','label')
trainDF, testDF = houseDF.randomSplit([0.8,0.2],seed=123)

lr = LinearRegression()
eva = RegressionEvaluator(metricName='rmse')
myParams = ParamGridBuilder().addGrid(lr.maxIter,[5,6,7])\
                    .addGrid(lr.elasticNetParam,[0.1,0.2,0.3]).build()
validator = CrossValidator(parallelism=4, 
                                 estimator=lr,
                                 estimatorParamMaps=myParams,
                                 evaluator=eva,
                                 numFolds=5)
model = validator.fit(trainDF)
print("Best Model hesaplandı")
print("Best maxIter : ", model.bestModel.getMaxIter())
print("Best elasticNetParam : ", model.bestModel.getElasticNetParam())

resultDF = model.transform(testDF)
result = eva.evaluate(resultDF)
print("Accuracy of the best model ",result)

Best Model hesaplandı
Best maxIter :  5
Best elasticNetParam :  0.1
Accuracy of the best model  3.952468980908394
